In [2]:
%matplotlib inline

Creates subject per voxel matrices to be used as features in between-subject classification.


In [3]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import nilearn
import scipy
import nibabel

from numpy import nan as NaN
from matplotlib import pyplot as plt
from scipy.stats.stats import pearsonr
from nilearn import image
from nilearn import plotting
from nilearn import datasets
from nilearn.plotting import plot_stat_map, plot_roi, plot_anat, plot_img, show
from nilearn.input_data import NiftiLabelsMasker
from nilearn.input_data import NiftiMasker
from nilearn.connectome import ConnectivityMeasure

Step 1. Create group mask 

In [7]:
# Load list of participants to include
s_list = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Participants/sub_list_TaskQC.tsv'
slist = pd.read_csv(s_list, sep = '\t')
print(slist.shape)
slist.set_index('sub_ids', inplace=True)
ids = slist.index
print(ids)

##Steps to create group mask here

#mask = mask 


# Mask images with group mask first? Can it concatenate images if number of vox differs per subject?


# for each contrast between conditions, load beta maps (one per subject, id in increasing order)
# Correct vs Incorrect Source
betas_cs_minus_ws = image.load_img(img='/Users/mombot/Documents/Simexp/CIMAQ/Data/test/testOut/Output/Betas/*/TaskContrasts/betas_sub*_cs_minus_ws.nii',
                           wildcards=True)

print(betas_cs_minus_ws.header.get_data_shape()) #shape of 4D betas map image object
print(betas_cs_minus_ws.header.get_zooms()) #voxel sizes in mm



#IMPORTANT: REMOVE SUBJECTS WHO DO NOT HAVE ANY TRIALS FROM EITHER CONDITION!!

# Other contrasts of interest:
#Encoding vs Control Condition
betas_enc_minus_ctl = image.load_img(img='/Users/mombot/Documents/Simexp/CIMAQ/Data/test/testOut/Output/Betas/*/TaskContrasts/betas_sub*_enc_minus_ctl.nii',
                           wildcards=True)

# Hit vs Miss
betas_hit_minus_miss = image.load_img(img='/Users/mombot/Documents/Simexp/CIMAQ/Data/test/testOut/Output/Betas/*/TaskContrasts/betas_sub*_hit_minus_miss.nii',
                           wildcards=True)

# Hit vs Control Condition
betas_hit_minus_ctl = image.load_img(img='/Users/mombot/Documents/Simexp/CIMAQ/Data/test/testOut/Output/Betas/*/TaskContrasts/betas_sub*_hit_minus_ctl.nii',
                           wildcards=True)

# Correct Source vs Miss
betas_cs_minus_miss = image.load_img(img='/Users/mombot/Documents/Simexp/CIMAQ/Data/test/testOut/Output/Betas/*/TaskContrasts/betas_sub*_cs_minus_miss.nii',
                           wildcards=True)

# Correct Source vs Control Condition
betas_cs_minus_ctl = image.load_img(img='/Users/mombot/Documents/Simexp/CIMAQ/Data/test/testOut/Output/Betas/*/TaskContrasts/betas_sub*_cs_minus_ctl.nii',
                           wildcards=True)


(94, 1)
Int64Index([108391, 120839, 122922, 127228, 139593, 147863, 150649, 164965,
            175295, 178101, 189005, 197192, 199801, 219637, 229301, 247659,
            254402, 255499, 258618, 258912, 267168, 270218, 271596, 314409,
            326073, 336665, 337021, 350555, 370092, 385370, 386333, 396250,
            403131, 408506, 413474, 427357, 437101, 439776, 441008, 458807,
            459801, 462345, 484204, 490035, 502616, 517070, 520377, 543589,
            549994, 555537, 567214, 597569, 619278, 628299, 630120, 652850,
            658178, 659068, 668786, 677561, 711830, 729722, 739694, 748676,
            763590, 778749, 783781, 785217, 785245, 804743, 845675, 866812,
            878354, 884343, 886007, 893978, 901551, 906145, 914042, 915022,
            920577, 932933, 936730, 938001, 955548, 956049, 956130, 968913,
            974246, 979001, 983291, 988602, 996599, 998166],
           dtype='int64', name='sub_ids')


In [ ]:
# use NiftiMasker object to convert epi images into a frames x voxels data matrix (310 x 69924)
masker = NiftiMasker(mask_img=mask, standardize=True)

cs_minus_ws = masker.fit_transform(betas_cs_minus_ws)

print(cs_minus_ws.shape) 

enc_minus_ctl = masker.fit_transform(betas_enc_minus_ctl)
hit_minus_miss = masker.fit_transform(betas_hit_minus_miss)
hit_minus_ctl = masker.fit_transform(betas_hit_minus_ctl)
cs_minus_miss = masker.fit_transform(betas_cs_minus_miss)
cs_minus_ctl = masker.fit_transform(betas_cs_minus_ctl)



In [ ]:
# Load latent memory score

npsych_data = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Participants/SubjectList.tsv'
neuropsych = pd.read_csv(npsych_data, sep = '\t')

neuropsych.set_index('participant_id', inplace=True)
neuropsych = neuropsych[neuropsych['QC_status'] is not 'F']
# neuropsych = neuropsych[neuropsych['QC_status'] != 'F']

# Latent memory score per participant
memoscore = neuropsych['Fac1_memory']

# Participant diagnosis 
status = neuropsych['cognitive_status']
